# Performance tips

## Topics
- Why Julia is fast
- LLVM compiler
- Parallellization

## Why is Julia fast?
- Rich type information, provided naturally by multiple dispatch
- Aggressive code specialization against run-time types
- JIT compilation using the LLVM compiler framework

In short, Julia is designated from the beginning to be fast. Not vice versa.

See the [scientific paper](https://arxiv.org/pdf/1209.5145v1.pdf) behind Julia, if you want to learn more.

## Optimizing nested loops: index "convention"

Julia is a column-major programming language. The inner loop should concern rows rather than columns. This is due to how arrays are stored in memory.

What this means is: **first index changes fastest**.
```julia
for k in 1:Nz
    for j in 1:Ny
        for i in 1:Nx
            arr[i,j,k]
        end
    end
end
```

In [ ]:
# This function is badly written, because it looks at every column of a row, then at 
# every column of the next row, and so on

function laplacian_bad(lap_x::Array{Float64,2}, x::Array{Float64,2})
    nr,nc = size(x)
    for ir = 2:nr-1 
        for ic = 2:nc-1 # bad loop nesting order
            lap_x[ir,ic] =
                (x[ir+1,ic] + x[ir-1,ic] +
                x[ir,ic+1] + x[ir,ic-1]) - 4*x[ir,ic]
        end
    end
end

In [ ]:
# In this version, the two loops are nested properly:

function laplacian_good(lap_x::Array{Float64,2}, x::Array{Float64,2})
    nr,nc = size(x)
    for ic = 2:nc-1
        for ir = 2:nr-1 # good loop nesting order
            lap_x[ir,ic] =
                (x[ir+1,ic] + x[ir-1,ic] +
                x[ir,ic+1] + x[ir,ic-1]) - 4*x[ir,ic]
        end
    end
end

## Let's see the effect in practise!

In [ ]:
using Printf
function main_test(nr, nc)
    field = zeros(nr, nc)
    for ic = 1:nc, ir = 1:nr
        if ir == 1 || ic == 1 || ir == nr || ic == nc
            field[ir,ic] = 1.0
        end
    end
    lap_field = zeros(size(field))

    time = @elapsed laplacian_bad(lap_field, field)
    @printf "laplacian_bad:          %.3f s\n" time
    
    time = @elapsed laplacian_good(lap_field, field)
    @printf "laplacian_good:         %.3f s\n" time

end
main_test(10^4, 10^4)

## Levels of parallellism
1. Instruction level parallellism
2. Vector instructions (see `Bonus_simd-vectorization.pynb` if you are interested)
3. **Threading** (shared-memory)
4. **Distributed**
5. Accelerators (e.g., GPGPU; *not covered here*)

## Advanced: A (very short) introduction to the interiors of Julia compiler

Let's stop treating our tools like blackboxes. Let's see what the compiler herself is thinking about our code with `using InteractiveUtils`.

1. `@code_lowered`
2. `@code_typed` and `@code_warntype`
3. `@code_llvm`
4. `@code_native`

See [slides](https://slides.com/valentinchuravy/julia-parallelism) by Valentin Churavy, for more.

In [ ]:
function mysum(A)
    acc = zero(eltype(A))
    for a in A
        acc += a
    end
    return acc
end

In [ ]:
using InteractiveUtils

In [ ]:
@code_lowered mysum(ones(1))

In [ ]:
@code_typed mysum(ones(1))

In [ ]:
@code_llvm mysum(ones(1))

In [ ]:
@code_native mysum(ones(1))

## About global scope
So what does the previous machine code actually mean? Well, in practise:

A global variable might have its value, and therefore its type, change at any given point. This makes it difficult/nigh impossible for the compiler to reason about/optimize code using global variables.

Julia uses functions as its compilation unit and any code that is performance critical or being benchmarked should be inside a function.

## Demo: Giving hints to the compiler 
Let's see what we can do with our previously defined `laplacian` function. 

For some aggressive cases we can provide the JIT-compiler `@inbounds` macro hinting that there is no need to check array bounds.

In [ ]:
#Lets re-define our previous Laplacians (from 05_ notebook)
function laplacian_bad(lap_x::Array{Float64,2}, x::Array{Float64,2})
    nr,nc = size(x)
    for ir = 2:nr-1, ic = 2:nc-1 # bad loop nesting order
        lap_x[ir,ic] =
            (x[ir+1,ic] + x[ir-1,ic] +
            x[ir,ic+1] + x[ir,ic-1]) - 4*x[ir,ic]
    end
end

#In this version, the two loops are nested properly:
function laplacian_good(lap_x::Array{Float64,2}, x::Array{Float64,2})
    nr,nc = size(x)
    for ic = 2:nc-1, ir = 2:nr-1 # good loop nesting order
        lap_x[ir,ic] =
            (x[ir+1,ic] + x[ir-1,ic] +
            x[ir,ic+1] + x[ir,ic-1]) - 4*x[ir,ic]
    end
end

In [ ]:
# A way to increase the speed is to remove the array bounds checking, using the macro @inbounds:
function laplacian_good_nocheck(lap_x::Array{Float64,2}, x::Array{Float64,2})
    nr,nc = size(x)
    for ic = 2:nc-1
        for ir = 2:nr-1 # good loop nesting order
            @inbounds begin lap_x[ir,ic] = # no array bounds checking
                (x[ir+1,ic] +  x[ir-1,ic] +
                x[ir,ic+1] + x[ir,ic-1]) - 4*x[ir,ic]
            end
        end
    end
end

In [ ]:
using Printf #evaluate me to get printf

In [ ]:
function main_test(nr, nc)
    field = zeros(nr, nc)
    for ic = 1:nc, ir = 1:nr
        if ir == 1 || ic == 1 || ir == nr || ic == nc
            field[ir,ic] = 1.0
        end
    end
    lap_field = zeros(size(field))

    time = @elapsed laplacian_bad(lap_field, field)
    @printf "laplacian_bad:          %.3f s\n" time
    
    time = @elapsed laplacian_good(lap_field, field)
    @printf "laplacian_good:         %.3f s\n" time
    
    time = @elapsed laplacian_good_nocheck(lap_field, field)
    @printf "laplacian_good_nocheck: %.3f s\n" time
end

main_test(10^4, 10^4)

## Threading (experimental)
Julia threading model is based on a fork-join approach and is still considered experimental.

Fork-join describes the control flow that a group of threads undergoes. Execution is then forked and an anonymous function is ran across all threads.

All threads have to join together and serial execution continues.

## Threading in practise
The number of threads Julia starts up with is controlled by an environment variable called `JULIA_NUM_THREADS`. Now, let's start up Julia with 4 threads:

```bash
export JULIA_NUM_THREADS=4
julia
```

NOTE: this does not work in the notebook environment because the kernel is automatically loaded with only 1 thread.

In [ ]:
using Base.Threads
nthreads()

## Using threads

```julia
@threads for id in 1:nthreads()
    #each thread does something
end
```

In [ ]:
a = zeros(10)
@threads for i = 1:10
    a[i] = Threads.threadid()
end
a

## Advanced: Threaded sum
Here is a more complex example of threaded sum from https://github.com/stevengj/18S096/blob/master/lectures/lecture5/Parallelism.ipynb

In [ ]:
function threaded_sum(arr)
   @assert length(arr) % nthreads() == 0
    
   let results = zeros(eltype(arr), nthreads())
       @threads for tid in 1:nthreads()
           # split work
           acc = zero(eltype(arr))
           len = div(length(arr), nthreads())
           domain = ((tid-1)*len +1):tid*len
           @inbounds for i in domain
               acc += arr[i]    
           end
           results[tid] = acc
       end
       sum(results)
   end
end

## Distributed computing 
Distributed processing uses individual processes that communicate with each other. In this case, data movement and communication is explicit!

Julia supports various forms of distributed computing. 
- **A native master-worker system based on remote procedure calls**
- MPI through [MPI.jl](https://github.com/JuliaParallel/MPI.jl)
- [DistributedArrays.jl](https://github.com/JuliaParallel/DistributedArrays.jl)

## Master-Worker model
We need to launch Julia with 
```bash
julia -p 4
```
then inside Julia you can check
```julia
nprocs()
workers()
```
which should print `5` and `[2,3,4,5]`. 

Why 5, you ask? Because *"worker 1"* is the *"boss"*. And bosses don't work.

Functions (and everything used by workers) needs to be explicitly declared for all:
```julia
@everywhere g(x) = 2x
```
Only then can we send the job to somebody else and fetch the result
```julia
remotecall_fetch(g, 3, 2.0)
```
Here we fetch the result of `g` of worker `3` applied to a value of `2.0`.

Use `@everywhere` to execute a top-level block on each process
```julia
@everywhere begin
    using Test
    include("src.jl")
end
```
Define variables on all processes
```julia
@everywhere bar = 1
```


## `@distributed` as a shortcut 
A parallel for loop of the form (from `using Distributed`):
```julia
@distributed [reducer] for var = range
    body
end
```
The specified range is partitioned and locally executed across all workers. In case an optional reducer function is specified, `@distributed` performs local reductions on each worker with a final reduction on the calling process.

Note that without a reducer function, `@distributed` executes asynchronously, i.e., it spawns independent tasks on all available workers and returns immediately without waiting for completion. To wait for completion, prefix the call with `@sync`, like :
```julia
@sync @distributed for var = range
      body
end
```

In [ ]:
using Distributed

nheads = @distributed (+) for i=1:200000000
  Int(rand(Bool))
end

## pmap for unbalanced load
In some cases no reduction operator is needed, and we merely wish to apply a function to all integers in some range. This is another useful operation called parallel map. 

For example, we could compute the singular values of several large random matrices in parallel as follows:

In [ ]:
using LinearAlgebra #loading svd()

In [ ]:
M = Matrix{Float64}[rand(1000,1000) for i=1:10]
pmap(svd, M)

`pmap()` is designed for the case where each function call does a large amount of work. In contrast, `@distributed for` can handle situations where each iteration is tiny, perhaps merely summing two numbers.

### Exercise 1: π in parallel

Let's compute the value of π=3.1415926.... in parallel using Monte Carlo simulations.

Below is a function that does this by throwing darts inside a box of $[-1, 1]^2$. It follows from the definition of π that the area fo a circle is $A = \pi r^2$, where $r$ is the radius of a circle. We develop a Monte Carlo algorithm to compute π by randomly throwing darts (i.e., generating uniformly distributed random numbers inside a unit box) and counting the fraction of the points that land inside the circle.

Let's try to visualize the setup first. 

In [ ]:
using Plots
pyplot()

plt = plot( xlims=(-2,2), ylims=(-2,2), aspect_ratio=1)
for n in 1:500
    x = rand()*2 - 1
    y = rand()*2 - 1
    
    r = sqrt(x^2 + y^2)
    if r < 1
        #inside
        scatter!([x], [y], color="red", label="")
    else
        #outside
        scatter!([x], [y], color="blue",label="")
    end
end
plt #display plot

Can you see the circle? Now, let's do some actual computing.

In [ ]:
function compute_pi(N::Int)
    n_landed_in_circle = 0  # counts number of points that have radial coordinate < 1, i.e. in circle
    for i = 1:N
        x = rand() * 2 - 1  # uniformly distributed number on x-axis
        y = rand() * 2 - 1  # uniformly distributed number on y-axis

        r = sqrt(x*x + y*y)  # radius squared, in radial coordinates
        if r < 1
            n_landed_in_circle += 1
        end
    end

    return n_landed_in_circle / N * 4.0    
end


The time macro is a quick way to benchmark the performance of our code.

In [ ]:
@time compute_pi(10^9)

#### Actual exercise
Your mission? Parallellize the `compute_pi` function! 

Hint: see the `for` loop? Remember `@distributed`? Remember that when using `@distributed` the result of each iteration is taken as the value of the last expression inside the loop. Therefore if your loops ends in
```julia
for 
    #blaablaa
    #...
    if something
        1
    else
        0
    end
end
```
it will result in returning either `1` or `0`.

You can also try adapting this to use `pmap` if you like. 

Remember to add workers before we run our sweet new function with this command:

In [ ]:
using Distributed
addprocs(4)
nprocs()

Finally, after you are done, see the full story at [Parallel Monte Carlo in Julia](http://corysimon.github.io/articles/parallel-monte-carlo-in-julia/)

### Advanced Exercise 1: Distributed Arrays

Install `DistributedArrays` as
```julia
Pkg.add("DistributedArrays")
```

then try them out as
```julia
addprocs(4) #adding 4 workers to share the load
@everywhere using DistributedArrays #loading DAs for every worker
A = fill(1.1, (100, 100) ) #create array
DA = distribute(A) #distribute it to workers
sum(DA)
```

### Advanced Exercise 2: MPI

Install `MPI` package by running:
```julia
Pkg.update()
Pkg.add("MPI")
```
In case of problems, see the [readme](https://github.com/JuliaParallel/MPI.jl)

Because of how MPI works, we need to explicitly write our code into a file. Create `01-hello.jl` and `01-hello-impl.jl` as follows:

`01-hello.jl` should look like this:
```julia

import MPI
include("01-hello-impl.jl")

function main()
    MPI.Init()

    do_hello()

    MPI.Finalize()
end

main()
```
and the actual implementation file `01-hello-impl.jl` like this
```julia

function do_hello()
    comm = MPI.COMM_WORLD
    println("Hello world, I am $(MPI.Comm_rank(comm)) of $(MPI.Comm_size(comm))")
    MPI.Barrier(comm)
end
```


You can execute your code the normal way as
```bash
mpirun -np 3 julia 01-hello.jl
```

See the MPI.jl [examples](https://github.com/JuliaParallel/MPI.jl/tree/master/examples) for more.

## Summary: General optimization tricks

- Write functions!
- Avoid global variables
    - A global variable might have its value, (and type) change at any given point. This makes it hard for the compiler to optimize.